In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from api_keys import g_key

In [2]:
cities_df = pd.read_csv("../cities_df.csv")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cheremshan,54.6667,51.5000,56.19,49,40,3.40,RU,1627249903
1,Rikitea,-23.1203,-134.9692,71.01,67,19,4.70,PF,1627249904
2,Kalmunai,7.4167,81.8167,80.02,79,100,4.85,LK,1627249904
3,Sinnamary,5.3833,-52.9500,78.53,82,81,6.64,GF,1627249905
4,Kazanskoye,55.6448,69.2350,51.78,79,1,9.73,RU,1627249905


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
clean_df=cities_df.loc[cities_df['Max Temp']>75]
clean_df=clean_df.loc[clean_df['Max Temp']<85]
clean_df=clean_df.loc[clean_df['Wind Speed']<10]
clean_df=clean_df.loc[clean_df['Humidity']<50]
clean_df=clean_df.loc[clean_df['Cloudiness']<25]
clean_df['Nearest Hotel']=''
clean_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel
8,Aripuanã,-9.1667,-60.6333,84.67,39,18,1.41,BR,1627249907,
74,Birjand,32.8663,59.2211,82.87,12,0,4.61,IR,1627249940,
110,Tiznit Province,29.5833,-9.5000,78.33,35,0,3.49,MA,1627249959,
255,Alta Floresta,-9.8756,-56.0861,82.38,35,18,2.46,BR,1627250036,
259,Yangi Marg`ilon,40.4272,71.7189,80.02,36,0,6.91,UZ,1627250038,
307,Pontes e Lacerda,-15.2261,-59.3353,83.61,30,13,3.29,BR,1627250063,
344,Korla,41.7597,86.1469,83.91,14,0,8.84,CN,1627250083,
359,Ürümqi,43.8010,87.6005,79.75,26,0,8.95,CN,1627250091,
374,Kumul,42.8000,93.4500,83.19,21,5,8.77,CN,1627250099,
377,Diamantino,-14.4086,-56.4461,81.81,33,14,5.79,BR,1627250100,


In [29]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "keyword":"hotel",
    #"radius":5000,
    "rankby": "distance",
    "type": "lodging",
    "key": g_key,
}
value=0
# use iterrows to iterate through pandas dataframe
for index, row in clean_df.iterrows():
    
    city =row["City"]
    lat = row['Lat']
    lng=row["Lng"]

    # add keyword to params dict
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {value}:")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"The closest hotel to {city} is {results[0]['name']}.")
        
        clean_df.loc[index, 'Nearest Hotel'] = results[0]['name']
        value=value+1
        
    except (KeyError, IndexError):
        print(f"No hotels within your radius for {city}")
        value=value+1
    print("------------")

Retrieving Results for Index 0:
No hotels within your radius for Aripuanã
------------
Retrieving Results for Index 1:
The closest hotel to Birjand is Birjand Tourist Hotel.
------------
Retrieving Results for Index 2:
The closest hotel to Tiznit Province is Hotel kerdous.
------------
Retrieving Results for Index 3:
The closest hotel to Alta Floresta is Lisboa Palace Hotel.
------------
Retrieving Results for Index 4:
The closest hotel to Yangi Marg`ilon is Diyor hotel.
------------
Retrieving Results for Index 5:
The closest hotel to Pontes e Lacerda is Plaza Hotel.
------------
Retrieving Results for Index 6:
The closest hotel to Korla is Ross Quest Hotel.
------------
Retrieving Results for Index 7:
The closest hotel to Ürümqi is Hongshan New Century Commercial Hotel.
------------
Retrieving Results for Index 8:
The closest hotel to Kumul is Sunshine Hotel & Resort.
------------
Retrieving Results for Index 9:
The closest hotel to Diamantino is Hotel Querencia.
------------
Retriev

In [34]:
nan_value = float("NaN")
clean_df.replace("", nan_value, inplace=True)
clean_df.dropna(subset = ["Nearest Hotel"], inplace=True)
clean_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearest Hotel
74,Birjand,32.8663,59.2211,82.87,12,0,4.61,IR,1627249940,Birjand Tourist Hotel
110,Tiznit Province,29.5833,-9.5000,78.33,35,0,3.49,MA,1627249959,Hotel kerdous
255,Alta Floresta,-9.8756,-56.0861,82.38,35,18,2.46,BR,1627250036,Lisboa Palace Hotel
259,Yangi Marg`ilon,40.4272,71.7189,80.02,36,0,6.91,UZ,1627250038,Diyor hotel
307,Pontes e Lacerda,-15.2261,-59.3353,83.61,30,13,3.29,BR,1627250063,Plaza Hotel
344,Korla,41.7597,86.1469,83.91,14,0,8.84,CN,1627250083,Ross Quest Hotel
359,Ürümqi,43.8010,87.6005,79.75,26,0,8.95,CN,1627250091,Hongshan New Century Commercial Hotel
374,Kumul,42.8000,93.4500,83.19,21,5,8.77,CN,1627250099,Sunshine Hotel & Resort
377,Diamantino,-14.4086,-56.4461,81.81,33,14,5.79,BR,1627250100,Hotel Querencia
392,São João do Piauí,-8.3581,-42.2467,80.38,40,3,8.41,BR,1627250109,Hotel Esplanada


In [66]:
locations = clean_df[["Lat", "Lng"]].astype(float)
hotel_list=clean_df["Nearest Hotel"].to_list()
country_list=clean_df["Country"].to_list()
city_list=clean_df["City"].to_list()

In [80]:
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    #info_box_content=[f"Nearest hotel in {row['City']},{row['Country']} is {row['Nearest Hotel']}" for row in clean_df]
    info_box_content=[
        f"Nearest Hotel: {hotel}, Country: {country}, City: {city}"
        for hotel, country, city
        in zip(hotel_list, country_list, city_list)
    ]
    #[f"Nearest Hotel: {hotel[9]} " for hotel in clean_df]
    #f"{c}," for c in country_list]
    #print(f"{city}") for city in city_list]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))